# Variables

This page provides an overview of tricks and tools for working with variables, including environment variables, in Linux.

## Nested shells

Some utilities are designed to operate based on specific values of environment variables. It can be frustrating at first when a variable you’ve defined isn’t visible to an invoked shell. Here are ways to pass variables to a nested shell:

- Define it before the command using the syntax `<var1>=<value1> <var2>=<value2> ... <var_n>=<value_n> <shell command>`.  
- Define the variables as environment variables using the `export` command.  


---

Consider `LOCAL_VARIABLE` - within this shell, you can access it freely. The following cell demonstrates how it is substituted into the output of `echo`:

In [1]:
LOCAL_VARIABLE="hello world"
echo $LOCAL_VARIABLE

hello world


However, the following cell attempts to access `LOCAL_VARIABLE` from another shell:

In [2]:
bash -c "echo \$LOCAL_VARIABLE"

There is no result — by executing `bash`, you have created another shell where `LOCAL_VARIABLE` is not defined.

The same issue with python.

In [3]:
python3 -c "import os; print(os.environ[\"LOCAL_VARIABLE\"])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'LOCAL_VARIABLE'


: 1

But if you prefix the shell invocation with the variable definition, everything works fine in the nested shell.

In [3]:
LOCAL_VARIABLE="hello variable" bash -c "echo \$LOCAL_VARIABLE"
LOCAL_VARIABLE="hello variable" python3 -c "import os; print(os.environ[\"LOCAL_VARIABLE\"])"

hello variable
hello variable


Or prefixing it with `export` will create this variable as an environment variable:

In [4]:
export ENVIRONMENT_VARIABLE="hello global"
echo $ENVIRONMENT_VARIABLE

hello global


Now it can be easily accessed from any nested shell.

In [5]:
python3 -c "import os; print(os.environ[\"ENVIRONMENT_VARIABLE\"])"
bash -c "echo \$ENVIRONMENT_VARIABLE"

hello global
hello global


## Keep variables in file

For convenince you can keep variables in file - typical name for such file is `.env`. In fact it is just script that have to be executed in the shell where you wan't to define varables - so you have just write command simply like that you can write in the shell, result would be the same.

To load variable from file you can use `. <file>` or `source <file>`.

---

As an example, consider a file created with the following code:

In [15]:
cat << EOF > /tmp/env
VARIABLE1="variable 1 value"
VARIABLE2="variable 2 value"
export ENVIRONMENT_EXAMPLE="environment hello"
EOF

After loading this file, you can use the variables defined in it as usual.

In [17]:
source /tmp/env
echo $VARIABLE1 $VARIABLE2
env | grep EXAMPLE
unset VARIABLE1 VARIABLE2 ENVIRONMENT_EXAMPLE

variable 1 value variable 2 value
ENVIRONMENT_EXAMPLE=environment hello


The same output with using `.` instead of `source`.

In [18]:
. /tmp/env
echo $VARIABLE1 $VARIABLE2
env | grep EXAMPLE
unset VARIABLE1 VARIABLE2 ENVIRONMENT_EXAMPLE

variable 1 value variable 2 value
ENVIRONMENT_EXAMPLE=environment hello


**Note:** In this context, you can't use `bash` to execute files that define variables, as they will only be defined in the invoked shell, not in the calling shell.

In [23]:
bash /tmp/env
echo $VARIABLE1 $VARIABLE2

## Unset variable

If you want to remove a variable from the current shell or delete a local variable, use the `unset <variable>` command.

---

The following cell creates a shell variable and an environment variable. Once created, you can use them in any command you like.

In [36]:
shell_var="hello shell"
export env_var="hello environment"

echo "$shell_var $env_var"

hello shell hello environment


But after applying `unset` to the variables, `echo` returns an empty output again.

In [37]:
unset shell_var env_var

echo "$shell_var $env_var"

## Replace env. var. (envsubst)

The `envsubst` utility is used to replace placeholders within a "template" with corresponding values from environment variables. It allows you to substitute variables in specific locations with their actual values.

---

In the following cell is created a template:

In [ ]:
cat << EOF > envsubst_example
User \${username} succesfully login his age is \${userage}.
EOF

If we print this template as it is - it will have `${username}` and `${userage}` just as text.

In [ ]:
cat envsubst_example

User ${username} succesfully login his age is ${userage}.


If we define the corresponding values `username` and `userage` and pass the file to the envsubst command, we will obtain a line with the substituted values.

In [ ]:
export username=Fedor
export userage=23
envsubst < envsubst_example
rm envsubst_example

User Fedor succesfully login his age is 23.
